# Research on DFB lasers  
---
目标 : 用Meep仿真出DFB的结构和给与源之后的结果
- 仿真Bragg Reflection
- 仿真QCL的材料layer结构
- 仿真光栅结构
- 组合查看仿真结果
- extra：mode intensity


## 基本概念介绍以及目的需求分析
---

### What is DFB?
- DFB 是 Distributed Feedback Laser（分布式反馈激光器）的缩写，是一种非常重要的单频激光器结构，广泛应用于光通信、光谱分析、传感和激光雷达等领域。
- DFB 激光器是一种在整个增益介质中内置周期性光栅，实现光反馈的激光器结构。
- 关键机制： Bragg Reflection 满足布拉格条件时，特定波长的光会被光栅强烈反射，并在有源区形成共振腔体，形成激光发射。
- 关于Bragg Reflection的相关机制探索，放在了另外一个notebook中。
> https://www.bilibili.com/video/BV1LR4y1G7aW/?spm_id_from=333.337.search-card.all.click&vd_source=94a597b5fbd359a011fb548c5a51fb78

### Why can use meep?
- Meep（MIT Electromagnetic Package）是一个开源的时域电磁波仿真软件，基于 FDTD 方法（Finite-Difference Time-Domain） 实现。
- 由于光具有波动性和粒子性，所以光可以看作是一种电磁波，满足麦克斯韦方程。
- 光的传播、干涉、反射、折射等都是波动现象。
- FDTD 将时间和空间离散，把麦克斯韦方程变成差分方程，在每一个时间步上逐步演化电场E和磁场H，得到时域中完整的光传播动画。

### What parameters of the material and grating structure should I use in meep and why?
| 参数                          | 必要性 | 含义                | Meep 中的字段                                          |
| --------------------------- | --- | ----------------- | -------------------------------------------------- |
| ✅ 折射率 `n` 或介电常数 `ε`         | 必须  | 光速变化与波传播方向的关键参数   | `index=` 或 `epsilon=`                              |
| ⚠️ 导电率 `σ`                  | 可选  | 模拟吸收材料的能量损耗       | `D_conductivity=`                                  |
| ⚠️ 色散参数（Drude / Lorentz 模型） | 可选  | 模拟频率相关的 ε（真实材料色散） | `LorentzianSusceptibility` / `DrudeSusceptibility` |
| ⚠️ 磁导率 `μ`（一般 = 1）          | 可选  | 对磁性材料建模           | `mu=`                                              |
| ⚠️ χ(2), χ(3) 非线性项          | 可选  | 非线性响应，如倍频、自聚焦     | 手动实现 via polarization updates                      |
| ⚠️ 坐标相关的 ε（各向异性）            | 可选  | 双折射或光子晶体中常见       | 用 `epsilon_diag` 或张量指定                             |


## 具体实现以及实验
---

### 材料定义方式以及堆叠方式

#### QCL layers stack


In [ ]:
import meep as mp
import numpy as np
from meep.materials import Ti,Au,GaAs

resolution = 50
sx = 12
sy = 5
cell = mp.Vector3(sx, sy, 0)
pml_layers = [mp.PML(0.1)]

# 各层厚度（单位：μm）
padge_thickness = 0.08
ngaas_thickness = 0.1
ti_thickness = 0.02
au_thickness = 0.18
core_thickness = 2.5
substrate_thickness = 1.0

# 材料定义（近似 ε = n^2）
si_gaas = mp.Medium(epsilon=12.9)
qcl_core = mp.Medium(epsilon=13.0)
pdge = mp.Medium(epsilon=10.0)


geometry = []

# Substrate
geometry.append(mp.Block(
    material=si_gaas,
    size=mp.Vector3(mp.inf, substrate_thickness),
    center=mp.Vector3(y=-sy/2 + substrate_thickness/2)
))

# QCL active region
geometry.append(mp.Block(
    material=qcl_core,
    size=mp.Vector3(mp.inf, core_thickness),
    center=mp.Vector3(y=-sy/2 + substrate_thickness + core_thickness/2)
))

# 上层金属叠层
y0 = -sy/2 + substrate_thickness + core_thickness
geometry += [
    mp.Block(material = pdge, size = mp.Vector3(mp.inf, padge_thickness), center = mp.Vector3(y=y0 + padge_thickness/2)),
    mp.Block(material = GaAs, size = mp.Vector3(mp.inf, ngaas_thickness), center = mp.Vector3(y=y0 + padge_thickness + ngaas_thickness/2)),
    mp.Block(material = Ti, size = mp.Vector3(mp.inf, ti_thickness), center = mp.Vector3(y=y0 + padge_thickness + ngaas_thickness + ti_thickness/2)),
    mp.Block(material = Au, size = mp.Vector3(mp.inf, au_thickness), center = mp.Vector3(y=y0 + padge_thickness + ngaas_thickness + ti_thickness + au_thickness/2))
]


# Source
sources = [mp.Source(
    src=mp.GaussianSource(frequency=1/0.1, fwidth=0.2),
    component=mp.Ez,
    center=mp.Vector3(-3, -sy/2 + 0.5),
    size=mp.Vector3(0, 1.0)
)]

tran_fr = mp.FluxRegion(center=mp.Vector3(6))
frequencies = np.linspace(10,10,100)

# Simulation setup
sim = mp.Simulation(
    cell_size=cell,
    resolution=resolution,
    boundary_layers=pml_layers,
    geometry=geometry,
    sources=sources,
    dimensions=2
)

sim.plot2D()


### 非周期性光栅定义方式

#### Grating structure

In [ ]:
import meep as mp

resolution = 50
sx = 12
sy = 10
sz = 30
cell = mp.Vector3(sx,0,sz)
pml_layers = [mp.PML(0.1)]

# 各层厚度（单位：μm）
sigaas_thickness = 5

# 材料定义（近似 ε = n^2）
si_gaas = mp.Medium(epsilon=12.9)

geometry = []

# Substrate
geometry.append(mp.Block(
                    material=si_gaas,
                    size=mp.Vector3(sigaas_thickness,sy,sz),
                    center=mp.Vector3(x = -sx/2 + sigaas_thickness/2)))

# 光栅参数
Λ = 0.5                    # 基本周期（μm）
slot_width = 0.1           # 槽宽（沿 X）
slot_depth = 5           # 镂刻深度（沿 Y）
slot_spacing = Λ / 2       # 间距
pattern = "|-||-|-||||||-|||||-||||-|||-||-||||"  # 狭缝排列样式

# 起始 X 位置，居中布置
z_start = -len(pattern) * slot_spacing / 2

# 放置高度（x 方向中心）
x_grating = sx - slot_depth / 2 

# 光栅 geometry 列表
grating_geometry = []

for i, symbol in enumerate(pattern):
    if symbol == "|":
        z_pos = z_start + i * slot_spacing
        grating_geometry.append(mp.Block(
                                material=mp.air,
                                size=mp.Vector3(slot_depth,sy,slot_width),
                                center=mp.Vector3(0,0,z_pos)))
    else:
        z_pos = z_start + i * slot_spacing * 0.5

geometry += grating_geometry

sim = mp.Simulation(
    cell_size=cell,
    resolution=resolution,
    boundary_layers=pml_layers,
    geometry=geometry,
    dimensions=2
)

sim.plot2D()

### Combine layer stack and grating structure


In [ ]:
import meep as mp
import numpy as np
from meep.materials import Au
import matplotlib.pyplot as plt

# basic setup (1 μm)
resolution = 50
sx = 40
sy = 0
# sz = 120
# cell = mp.Vector3(sx, sy, sz)
frequency = 1e12

# thickness of different layers（1 μm）
au_thickness = 0.18
ngaas_thickness_1 = 0.08
Active_region_thickness = 0.0113
ngaas_thickness_2 = 0.7
sigaas_thickness = 30

# material definition
Active_region = mp.air
# Au = mp.Medium(epsilon=1.0, D_conductivity=1e7)
GaAs_low_doped = mp.Medium(epsilon=12.9, D_conductivity=0)
GaAs_high_doped = mp.Medium(epsilon=5, D_conductivity=100)

n_GaAs = mp.Medium(
    epsilon=10.9,  # 高频极限介电常数
    E_susceptibilities=[
        mp.DrudeSusceptibility(
            frequency=0.37,   # MEEP单位下的等离子体频率
            gamma=0.01,       # 自定义一个经验值（越小Q值越高）
            sigma=1.0         # 电导率（可调节匹配实验）
        )
    ]
)

geometry = []
# Substrate
geometry.append(mp.Block(
                    material=GaAs_low_doped,
                    size=mp.Vector3(sigaas_thickness,mp.inf,sz),
                    center=mp.Vector3(x = -sx/2 + sigaas_thickness/2)))

geometry.append(mp.Block(
                    material=GaAs_high_doped,
                    size=mp.Vector3(ngaas_thickness_2,mp.inf,sz),
                    center=mp.Vector3(x= -sx/2 + sigaas_thickness + ngaas_thickness_2/2)))

# QCL active region
geometry.append(mp.Block(
                    material=Active_region,
                    size=mp.Vector3(Active_region_thickness,mp.inf,sz),
                    center=mp.Vector3(x= -sx/2 + sigaas_thickness + ngaas_thickness_2 + Active_region_thickness/2)))

# upper layers
x0 = -sx/2 + sigaas_thickness + ngaas_thickness_2 + Active_region_thickness
geometry += [
    mp.Block(material = GaAs_high_doped, 
             size = mp.Vector3(ngaas_thickness_1,mp.inf,sz), 
             center = mp.Vector3(x = x0 + ngaas_thickness_1/2)),
    mp.Block(material = Au,    
             size = mp.Vector3(au_thickness,mp.inf,sz),    
             center = mp.Vector3(x = x0 + ngaas_thickness_1 +  au_thickness/2))]
    
# grating parameters
# Λ = 1                 
# slot_width = 0.2           
# slot_depth = 10
# slot_spacing = Λ / 2       
# pattern_1 = "|-||-|-||||||-|||||-||||-|||-||-||||-|||||||-||-||-|||-||-|-||-|||-||-|-||||||-||-|-|||-||-|||||-|||-||-|"
# pattern_2 = "|-||-|||||-||||||-|||-||||-||-||-|||||||||-||-|-||-||-||-|-|||-||-||-|-||||||-||||||||-||-||||||-||||---|"
# pattern = pattern_1 + pattern_2

# # 起始 z 位置，居中布置
# z_start = -len(pattern) * slot_spacing / 2

# # 放置高度（x 方向中心）
# x_grating = x0 + ngaas_thickness_1 +  au_thickness/2

# # 光栅 geometry 列表
# grating_geometry = []
# for i, symbol in enumerate(pattern):
#     if symbol == "|":
#         z_pos = z_start + i * slot_spacing
#         grating_geometry.append(mp.Block(material=mp.air,
#                                          size=mp.Vector3(slot_depth,mp.inf,slot_width),
#                                          center=mp.Vector3(x_grating,0,z_pos)))
#     # else:
#     #     z_pos = z_start + i * slot_spacing * 0.5

# geometry += grating_geometry

###-------------------------------------------------------------grating parameters---------------------------------------------------------------------###
pattern_1 = "|-||-|-||||||-|||||-||||-|||-||-||||-|||||||-||-||-|||-||-|-||-|||-||-|-||||||-||-|-|||-||-|||||-|||-||-|"
pattern_2 = "|-||-|||||-||||||-|||-||||-||-||-|||||||||-||-|-||-||-||-|-|||-||-||-|-||||||-||||||||-||-||||||-||||---|"
pattern = pattern_1 + pattern_2
slot_depth = 5
slot_width = 0.1
line = 1
dash = line * 1.5
sz_length = 0
for symbol in pattern:
    sz_length += line if symbol == '|' else dash

z_start = -sz_length/2
x_grating = x0 + ngaas_thickness_1 +  au_thickness/2
z_pos = -sz_length/2
def add_grating(pattern:list,grating_geometry:list):
    """ add grating structure through z axis"""
    global z_pos
    for symbol in pattern:
        z_pos = z_pos + line if symbol == "|" else z_pos + dash
        grating_geometry.append(mp.Block(
                            material=mp.air,
                            size=mp.Vector3(slot_depth,mp.inf,slot_width),
                            center=mp.Vector3(x_grating,0,z_pos)))

grating_geometry = []
add_grating(pattern,grating_geometry)
###---------------------------------------cell setup----------------------------------------------------###
sy = 0
sz = sz_length + 2 + 2
cell = mp.Vector3(sx, sy, sz)
geometry += grating_geometry
# Source
sources = [mp.Source(
    src=mp.ContinuousSource(frequency=frequency),  
    component=mp.Ez,
    center=mp.Vector3(0, 0,-sz/2),               
    size=mp.Vector3(10,0,0))]

# Simulation setup
sim = mp.Simulation(
    cell_size=cell,
    resolution=resolution,
    boundary_layers=[mp.PML(1, direction=mp.X),
                     mp.PML(1, direction=mp.Z)],
    geometry=geometry,
    sources=sources,
    dimensions=2)

sim.plot2D()
# sim.run(until = 200)

